In [14]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
from keras.utils.np_utils import to_categorical

# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

import pickle

import warnings
warnings.filterwarnings('ignore')

In [15]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [16]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)


df = describe_dataset("./dataset.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
C    8238
L    7134
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [17]:
# load dataset
df = pd.read_csv("./dataset.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 15372 
Number of columns: 37

Labels: 
0    8238
1    7134
Name: label, dtype: int64



In [18]:
# Standard Scaling of features
sc = StandardScaler()

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.fit_transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.25)

In [23]:
model = Sequential()
model.add(Dense(36, input_dim = 36, activation = "relu"))
model.add(Dense(18, activation = "relu"))
model.add(Dense(6, activation = "relu"))
model.add(Dense(2, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 36)                1332      
                                                                 
 dense_12 (Dense)            (None, 18)                666       
                                                                 
 dense_13 (Dense)            (None, 6)                 114       
                                                                 
 dense_14 (Dense)            (None, 2)                 14        
                                                                 
Total params: 2,126
Trainable params: 2,126
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))

Epoch 1/100


2022-10-21 00:34:37.787710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1153/1153 [==============================] - ETA: 0s - loss: 0.0579 - accuracy: 0.9819

2022-10-21 00:34:47.509209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1153/1153 [==============================] - 12s 10ms/step - loss: 0.0579 - accuracy: 0.9819 - val_loss: 0.0698 - val_accuracy: 0.9844
Epoch 2/100
1153/1153 [==============================] - 11s 10ms/step - loss: 0.0289 - accuracy: 0.9925 - val_loss: 0.0439 - val_accuracy: 0.9909
Epoch 3/100
1153/1153 [==============================] - 11s 10ms/step - loss: 0.0282 - accuracy: 0.9928 - val_loss: 0.0158 - val_accuracy: 0.9953
Epoch 4/100
1153/1153 [==============================] - 11s 9ms/step - loss: 0.0233 - accuracy: 0.9931 - val_loss: 0.0171 - val_accuracy: 0.9948
Epoch 5/100
1153/1153 [==============================] - 11s 9ms/step - loss: 0.0250 - accuracy: 0.9936 - val_loss: 0.0226 - val_accuracy: 0.9922
Epoch 6/100
1153/1153 [==============================] - 11s 9ms/step - loss: 0.0248 - accuracy: 0.9945 - val_loss: 0.0255 - val_accuracy: 0.9943
Epoch 7/100
1153/1153 [==============================] - 11s 9ms/step - loss: 0.0191 - accuracy: 0.9945 - val_loss: 0.0252 - val_accu

In [25]:
from sklearn.metrics import confusion_matrix

predict_x = model.predict(x_test) 
y_pred_class = np.argmax(predict_x, axis=1)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)

confusion_matrix(y_test_class, y_pred_class)

 88/121 [====================>.........] - ETA: 0s

2022-10-21 00:53:06.260531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


121/121 [==============================] - 0s 2ms/step


array([[2043,    4],
       [   2, 1794]])

In [26]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2047
           1       1.00      1.00      1.00      1796

    accuracy                           1.00      3843
   macro avg       1.00      1.00      1.00      3843
weighted avg       1.00      1.00      1.00      3843



In [27]:
# Dump the best model to a pickle file
with open("./model/bicep_model_deep_learning.pkl", "wb") as f:
    pickle.dump(model, f)



INFO:tensorflow:Assets written to: ram://1172770c-92ef-499a-9f6a-762c9d25b9f3/assets


In [28]:
# Dump input scaler
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)